# Connexion

In [1]:
import sqlite3
import pandas as pd 
from datetime import datetime
import dateutil.relativedelta

conn = sqlite3.connect('data/olist.db')

cursor = conn.cursor()

In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables dans la base de données :", tables)

Tables dans la base de données : [('customers',), ('geoloc',), ('order_items',), ('order_pymts',), ('order_reviews',), ('orders',), ('products',), ('sellers',), ('translation',)]


# First request

In [38]:
request = f"""SELECT order_id, order_delivered_customer_date, order_estimated_delivery_date FROM orders 
                WHERE order_status != 'canceled' 
                AND order_purchase_timestamp > DATE((SELECT MAX(order_purchase_timestamp) FROM orders), '-3 months')
                 AND julianday(order_delivered_customer_date) - julianday(order_estimated_delivery_date) >= 3"""
df = pd.read_sql_query(request, conn)
df

,order_id,order_delivered_customer_date,order_estimated_delivery_date
0,cfa4fa27b417971e86d8127cb688712f,2018-08-29 01:41:41,2018-08-22 00:00:00
1,234c056c50619f48da64f731c48242b4,2018-09-01 18:14:42,2018-08-23 00:00:00
2,8ad3f1d0f96992e43566c4c82c9f6c58,2018-08-14 04:04:40,2018-08-03 00:00:00
3,7f579e203c931f3e8410103359c6d523,2018-08-13 20:11:47,2018-08-09 00:00:00
4,cb6e441ff2ef574ce08d3709426f88ec,2018-08-18 01:11:58,2018-08-15 00:00:00
...,...,...,...
317,874a7690bc049bd4ce210d195bdfff7b,2018-08-13 15:50:48,2018-08-08 00:00:00
318,587e32dd528769d669d539531d32aeb5,2018-09-03 09:32:31,2018-08-28 00:00:00
319,3adb141ba4bd69dd7fe8d3fb733c6b74,2018-08-28 18:02:52,2018-08-24 00:00:00
320,4cf09d9e5ebbe0f91ddd7bf9aae891cd,2018-08-21 01:12:45,2018-08-10 00:00:00


# Second request

In [6]:
request = f"SELECT seller_id, sum(price) as CA FROM order_items GROUP BY seller_id HAVING CA > 100000"
df = pd.read_sql_query(request, conn)
df

,seller_id,CA
0,1025f0e2d44d7041d6cf58b6550e0bfa,138968.55
1,1f50f920176fa81dab994f9023523100,106939.21
2,46dc3b2cc0980fb8ec44634e21d2718e,128111.19
3,4869f7a5dfa277a7dca6462dcf3b52b2,229472.63
4,4a3ca9315b744ce9f8e9374361493884,200472.92
5,53243585a1d6dc2643021fd1853d8905,222776.05
6,5dceca129747e92ff8ef7a997dc4f8ca,112155.53
7,620c87c171fb2a6dd6e8bb4dec959fc6,114774.50
8,6560211a19b47992c3666cc44a7e94c0,123304.83
9,7a67c85e85bb2ce8582c35f2203ad736,141745.53


## third request

In [35]:
request = f"""SELECT count(B.seller_id) as orders_count, B.seller_id, MIN(A.order_purchase_timestamp) as min_date
                FROM (
                    SELECT * FROM orders
                    ) as A
                    LEFT JOIN (
                    SELECT order_id, seller_id FROM order_items ) as B on A.order_id = B.order_id
                    GROUP BY B.seller_id
                    HAVING min_date >= DATE((SELECT MAX(order_purchase_timestamp) FROM orders), '-3 months')
                    AND orders_count >= 30;"""
df = pd.read_sql_query(request, conn)
df

,orders_count,seller_id,min_date
0,36,240b9776d844d37535668549a396af32,2018-07-17 13:48:59
1,52,81f89e42267213cb94da7ddc301651da,2018-08-08 12:45:12
2,69,d13e50eaa47b4cbe9eb81465865d8cfc,2018-08-04 09:09:37


In [36]:
request = f"""SELECT count(B.order_item_id) as items_count, B.seller_id, MIN(A.order_purchase_timestamp) as min_date
                FROM (
                    SELECT * FROM orders
                    ) as A
                    LEFT JOIN (
                    SELECT order_id, seller_id, order_item_id FROM order_items ) as B on A.order_id = B.order_id
                    GROUP BY B.seller_id
                    HAVING min_date >= DATE((SELECT MAX(order_purchase_timestamp) FROM orders), '-3 months')
                    AND items_count >= 30;"""
df = pd.read_sql_query(request, conn)
df

,items_count,seller_id,min_date
0,36,240b9776d844d37535668549a396af32,2018-07-17 13:48:59
1,52,81f89e42267213cb94da7ddc301651da,2018-08-08 12:45:12
2,69,d13e50eaa47b4cbe9eb81465865d8cfc,2018-08-04 09:09:37


## 4th request

In [17]:
request = f"""SELECT AVG(A.review_score) as AVG_review, C.customer_zip_code_prefix, count(C.customer_zip_code_prefix) as count_zip
                FROM (
                    SELECT review_score, review_id, order_id, review_creation_date FROM order_reviews
                    ) as A
                LEFT JOIN (
                    SELECT order_id, customer_id, order_purchase_timestamp  FROM orders
                    WHERE order_purchase_timestamp >= DATE(
                    (SELECT MAX(order_purchase_timestamp) FROM orders), 
                    '-12 months')
                    ) as B on A.order_id = B.order_id
                LEFT JOIN(
                    SELECT customer_id, customer_zip_code_prefix FROM customers
                ) as C on B.customer_id = C.customer_id
                GROUP BY C.customer_zip_code_prefix
                HAVING count_zip > 30
                ORDER BY AVG_review
                LIMIT 5;"""
df = pd.read_sql_query(request, conn)
df

,AVG_review,customer_zip_code_prefix,count_zip
0,2.808511,22753,47
1,3.135135,22770,37
2,3.233333,22793,90
3,3.277778,21321,36
4,3.351351,22780,37


In [18]:
request = f"""SELECT AVG(A.review_score) as AVG_review, C.customer_zip_code_prefix, count(C.customer_zip_code_prefix) as count_zip
                FROM (
                    SELECT review_score, review_id, order_id, review_creation_date FROM order_reviews
                    WHERE review_creation_date >= DATE(
                    (SELECT MAX(review_creation_date) FROM order_reviews), 
                    '-12 months')
                    ) as A
                LEFT JOIN (
                    SELECT order_id, customer_id FROM orders
                    ) as B on A.order_id = B.order_id
                LEFT JOIN(
                    SELECT customer_id, customer_zip_code_prefix FROM customers
                ) as C on B.customer_id = C.customer_id
                GROUP BY C.customer_zip_code_prefix
                HAVING count_zip > 30
                ORDER BY AVG_review
                LIMIT 5;"""
df = pd.read_sql_query(request, conn)
df

,AVG_review,customer_zip_code_prefix,count_zip
0,2.867925,22753,53
1,3.000000,22723,31
2,3.125000,28893,32
3,3.184211,22770,38
4,3.272727,13056,33


## tables to csv

In [19]:
for table in tables:
    request = f"""SELECT * FROM {table[0]};"""
    df = pd.read_sql_query(request, conn)
    df.drop(columns=["index"], inplace=True)
    df.to_csv(f"data/{table[0]}.csv", index=False)